# Validate On Anomalies

In [4]:
import numpy as np
import pandas as pd

import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.absolute()
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))

from src.config import TA_MU, TA_SIGMA, SLP_MU, SLP_SIGMA

/Users/yannis/PycharmProjects/WeRec3D/src


# Compare Normalized Anomaly MAEs 

In [5]:
EXPERIMENTS = [
               # 'ex1_baseline',  # Is already anomaly 
               'ex2_seasonal_component',
               'ex3_incremental_pretraining',
               'ex3.1_moving_window',
               'ex3.2_cm_inclusion',
               'ex3.3_elevation',
               'ex3.4_pi_init',
               'ex4.1_elev_mov_win',
               'ex4.2_elev_cmi',
               'ex4.3_elev_pi_init',
               'ex5.1_elev_mov_cmi',
               'ex5.2_elev_mov_pi',
               'ex6.1_elev_mov_cmi_pi'
               ]

In [6]:
# Unit refers to °C or Pa
unit_val_ymean = np.load("../data_sets/ymean_sets/validation_ymean.npy")
unit_test_ymean = np.load("../data_sets/ymean_sets/test_ymean.npy")

normalized_gt = np.load("../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3640, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3640]

exp_mae_map = {}
for exp_name in EXPERIMENTS:
    normalized_pred = np.load(f"{exp_name}/prediction/predicted.npy")
    
    norm_pred_anom = normalized_pred - normalized_val_ymean
    norm_gt_anom = normalized_gt - normalized_val_ymean
    
    normalized_anomaly_mae = np.mean(np.abs(norm_pred_anom - norm_gt_anom)) 
    print(f"Experiment {exp_name}: {normalized_anomaly_mae}")
    exp_mae_map[exp_name] = normalized_anomaly_mae

Experiment ex2_seasonal_component: 0.1799773419831189
Experiment ex3_incremental_pretraining: 0.12206587481964659
Experiment ex3.1_moving_window: 0.13372053883321702
Experiment ex3.2_cm_inclusion: 0.12394932871814245
Experiment ex3.3_elevation: 0.12162982006473548
Experiment ex3.4_pi_init: 0.12237111319549918
Experiment ex4.1_elev_mov_win: 0.11522942602932976
Experiment ex4.2_elev_cmi: 0.12298176019046475
Experiment ex4.3_elev_pi_init: 0.12380379143756348
Experiment ex5.1_elev_mov_cmi: 0.11135882305657814
Experiment ex5.2_elev_mov_pi: 0.1168156227956957
Experiment ex6.1_elev_mov_cmi_pi: 0.11801612023727144


In [7]:
# Almost IDENTICAL to the climatology-MAE results! -> Seasonal component has been learned very well!
exp_mae_map

{'ex2_seasonal_component': 0.1799773419831189,
 'ex3_incremental_pretraining': 0.12206587481964659,
 'ex3.1_moving_window': 0.13372053883321702,
 'ex3.2_cm_inclusion': 0.12394932871814245,
 'ex3.3_elevation': 0.12162982006473548,
 'ex3.4_pi_init': 0.12237111319549918,
 'ex4.1_elev_mov_win': 0.11522942602932976,
 'ex4.2_elev_cmi': 0.12298176019046475,
 'ex4.3_elev_pi_init': 0.12380379143756348,
 'ex5.1_elev_mov_cmi': 0.11135882305657814,
 'ex5.2_elev_mov_pi': 0.1168156227956957,
 'ex6.1_elev_mov_cmi_pi': 0.11801612023727144}

In [13]:
df = pd.DataFrame(data={'exp_name': exp_mae_map.keys(), 'anomaly_mae': exp_mae_map.values()} )
df.head(20)

,exp_name,anomaly_mae
0,ex2_seasonal_component,0.179977
1,ex3_incremental_pretraining,0.122066
2,ex3.1_moving_window,0.133721
3,ex3.2_cm_inclusion,0.123949
4,ex3.3_elevation,0.121630
5,ex3.4_pi_init,0.122371
6,ex4.1_elev_mov_win,0.115229
7,ex4.2_elev_cmi,0.122982
8,ex4.3_elev_pi_init,0.123804
9,ex5.1_elev_mov_cmi,0.111359
